In [2]:
from transformers import BertTokenizer, BertModel

In [3]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
sample_text = "Hello nice to meet you"
inputs = bert_tokenizer(sample_text, return_tensors="pt")
print(inputs)
last_hidden = bert_model(**inputs)['last_hidden_state']

{'input_ids': tensor([[ 101, 7592, 3835, 2000, 3113, 2017,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [5]:
import json
from tqdm import tqdm
with open('dataset/dataset.json','rb') as op:
    captions = json.load(op)

In [9]:
train_text_embs = []
test_text_embs = []

for  image_captions in tqdm(captions['images']):
    text_features = []
    for sentence in image_captions['sentences']:
        inputs = bert_tokenizer(sentence['raw'], return_tensors="pt")
        last_hidden = bert_model(**inputs)['last_hidden_state'].data.cpu().numpy()
        text_features.append(last_hidden[0])
    if image_captions['split'] == 'test':
        test_text_embs.append(text_features)
    else:
        train_text_embs.append(text_features)
        

  6%|▌         | 1726/31014 [08:37<2:58:53,  2.73it/s]

In [21]:
import pickle
with open('dataset/bertTrain_text_embs.pkl', 'wb') as op:
    pickle.dump(train_text_embs, op)

with open('dataset/bertTest_text_embs.pkl', 'wb') as op:
    pickle.dump(test_text_embs, op)